In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCom6YhUY62jM52nIMjf5_dw/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 478


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

10:26
13:57
10:01
10:09
10:20
10:02
10:03
10:12
10:43
10:18
10:55
10:08
10:06
4:54
12:59
10:02
10:01
10:01
10:11
10:02
13:30
11:57
10:52
10:06
7:17
10:13
10:32
6:57
11:30
40:46
13:47
10:02
10:01
10:01
10:03
10:11
12:04
10:53
10:50
10:02
10:18
11:14
10:02
10:01
10:05
10:09
10:01
10:18
45:19
6:39
12:01
10:56
9:00
13:50
11:03
13:41
10:41
6:48
10:03
13:17
12:24
5:37
1:11:57
11:26
13:53
15:44
10:15
14:03
11:14
4:55
10:04
10:09
12:33
10:55
10:28
10:55
10:05
7:48
10:01
10:13
10:06
11:36
11:00
13:28
14:30
6:44
10:01
10:09
10:22
10:25
10:38
11:54
3:18
10:21
10:02
10:01
14:14
10:06
10:35
11:07
6:15
6:39
11:52
13:49
14:56
14:19
7:02
11:21
11:25
11:49
10:03
12:34
8:08
10:07
11:50
12:19
12:22
11:14
10:37
11:25
10:02
20:03
11:38
10:32
10:30
10:06
10:24
12:27
13:18
11:16
10:07
11:16
11:27
10:04
11:09
10:03
15:16
10:02
10:01
10:18
10:45
10:22
10:42
10:41
11:40
10:15
10:01
11:38
11:44
10:05
14:41
10:41
10:56
10:26
6:14
16:33
10:08
10:21
6:22
11:49
14:57
13:59
10:12
12:22
10:23
12:56
10:48
10:04
11:02
1

In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [ ]:
from selenium.common.exceptions import ElementNotInteractableException

for i in range(len(video_list2)):
    try:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
                                                        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except ElementNotInteractableException:

        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        # browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        # category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # category = html.find('div', {'id':'always-shown'}).find('div',{'id':'content'}).find("a":{"class":"style-scope"}).text
        category = html.select_one("div#always-shown div#content a.style-scope")
        
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except AttributeError:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        #likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        #                                                'aria-label':re.compile('좋아요')}).text+'개'
        #dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        #'aria-label':re.compile('싫어요')}).text+'개'
        likes_count = "0"
        dislikes_count = "0"
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
        
        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    
    finally:    
        thumbnail_list.append(thumbnail)
        title_list.append(title)
        category_list.append(category)
        view_count_list.append(view_count)
        likes_count_list.append(likes_count)
        dislikes_count_list.append(dislikes_count)
        upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/TfY55p-uHOg/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB6Si2aAG5DM0uRLkEPLDfC5pRQ6g 제목: 역대 최고 후원금 받았습니다. ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 83,106회 좋아요수 1.2천개 싫어요수 21개 게시일 2020. 1. 9.
1 . 썸네일 https://i.ytimg.com/vi/YSSKyJ3fDNY/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC-57MQf9txHyMB1MWSZrIDJqgcrw 제목: 여자친구에게 삥뜯겼었습니다...ㅣ윷놀이 썰 (2/2)ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 97,272회 좋아요수 1천개 싫어요수 17개 게시일 2020. 1. 8.
2 . 썸네일 https://i.ytimg.com/vi/x6lMPRN0kC0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDonPTcbAIvYAYHpOrPJtpa7KHRzw 제목: 이것은 저의 과거입니다 ㅣ윷놀이 편 (1/2)ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 96,248회 좋아요수 1.3천개 싫어요수 26개 게시일 2020. 1. 8.
3 . 썸네일 https://i.ytimg.com/vi/l5uktjcDmJk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB10vrSVgiV9Mc-nE1yoqhDDmHrIA 제목: 저는 바보입니다. 카테고리 엔터테인먼트 조회수 조회수 174,251회 좋아요수 2.9천개 싫어요수 62개 게시일 2020. 1. 7.
4 . 썸네일 https://i.ytimg.com/vi/v3S_3Bz1o90/hqdefaul

35 . 썸네일 https://i.ytimg.com/vi/U0_3HXbodPU/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDz_i_wHf-e4BffUezR7LhJXBfZVQ 제목: 여자친구가 인스타는 하는데 카톡은 안본다...? 카테고리 엔터테인먼트 조회수 조회수 405,909회 좋아요수 4.2천개 싫어요수 94개 게시일 2019. 12. 9.
36 . 썸네일 https://i.ytimg.com/vi/qz69LC4TDSg/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLArvQ0Wl28fd597sLVpBMY5-V0Fow 제목: 여자친구한테 '절대'해서 안되는 발언 l 오잉커플 썰(2/3) 카테고리 엔터테인먼트 조회수 조회수 298,284회 좋아요수 3.8천개 싫어요수 92개 게시일 2019. 12. 8.
37 . 썸네일 https://i.ytimg.com/vi/3OcDzJ_nBGc/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLABQG5V6NwClpsN-Tl2gi9jMxY6Pg 제목: 폭탄머리하고 첫 만남부터 연애까지 썰 품 l 오잉커플 썰(1/3) 카테고리 엔터테인먼트 조회수 조회수 292,909회 좋아요수 4.3천개 싫어요수 63개 게시일 2019. 12. 7.
38 . 썸네일 https://i.ytimg.com/vi/74E4avoBPSE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBdYYsis_QFohqARUocfQBBpBarRQ 제목: 멘토스 입에다 박아넣고 엄청 때리고싶네 l 지스타 썰(4/4) 카테고리 엔터테인먼트 조회수 조회수 129,276회 좋아요수 1.8천개 싫어요수 32개 게시일 2019. 12. 6.
39 . 썸

70 . 썸네일 https://i.ytimg.com/vi/DIeVEKrBANs/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDDUBCgQEa1_qWp1S55raAcVGkgmA 제목: 세기말 승급전 특징 카테고리 엔터테인먼트 조회수 조회수 144,426회 좋아요수 1.3천개 싫어요수 27개 게시일 2019. 11. 7.
71 . 썸네일 https://i.ytimg.com/vi/IKcKD0Z6dF4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC27i5vOea_rifORHtrlwNyhYW3dA 제목: 귀..귀...귀척하지마 뒤지고 싶지않으면 카테고리 엔터테인먼트 조회수 조회수 566,232회 좋아요수 5.6천개 싫어요수 91개 게시일 2019. 11. 7.
72 . 썸네일 https://i.ytimg.com/vi/WbgX8nek-xw/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCRmq84e4pEpbva3RhLa0T_76HWTQ 제목: 택배 보낸 사람 진짜 누구냐...?? 카테고리 엔터테인먼트 조회수 조회수 131,985회 좋아요수 2.5천개 싫어요수 26개 게시일 2019. 11. 6.
73 . 썸네일 https://i.ytimg.com/vi/TtQMT4iI43s/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCHbN5pEGO6cscYhvBdr8xOe0qaLw 제목: 경상도 분들 죄송합니다... 제발 화내지말아주세요. 카테고리 엔터테인먼트 조회수 조회수 213,712회 좋아요수 3천개 싫어요수 55개 게시일 2019. 11. 6.
74 . 썸네일 https://i.ytimg.com/vi/mQ7LNbD8kp4/hqdefault.jpg?

104 . 썸네일 https://i.ytimg.com/vi/xcZulRQEMk4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA7-PsrI-HGGt6VqZfYGkxAvzyyvA 제목: 새삼 느끼는 OGN의 위대함...   l OGN 황혼에서 새벽까지 (2/3) l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 32,082회 좋아요수 421개 싫어요수 3개 게시일 2019. 10. 10.
105 . 썸네일 https://i.ytimg.com/vi/5NHT1iYxNf8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAPkthAzBw1Rh_a7nGs0Wqqgz5pGw 제목: 힘조절 실패해서 하마터면 고소당할 뻔했습니다... 죄송합니다.  l OGN 황혼에서 새벽까지 (1/3) l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 54,096회 좋아요수 830개 싫어요수 13개 게시일 2019. 10. 10.
106 . 썸네일 https://i.ytimg.com/vi/nKHNFDzQBCU/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDQrtUE1E_e5IHo_l8s3SE9q6sjXg 제목: 크게 한탕할 수 있는 고수익 알바 소개해드립니다! l 개그신동 썰 (3/3) l 오킹TV l 카테고리 엔터테인먼트 조회수 조회수 122,226회 좋아요수 1.7천개 싫어요수 19개 게시일 2019. 10. 10.
107 . 썸네일 https://i.ytimg.com/vi/CKFQftsBE3c/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCOy0s32c8aqIFl8iYVO3PXC4FFqA 제목: 전국민을 앞에 두고 갑분싸 일으켜서 죄송합니다... l 개그신동 썰

136 . 썸네일 https://i.ytimg.com/vi/4I9yhOls7Lw/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBRxEAUukmpiYlf_kwlg6EuPA-cJQ 제목: 역대급 구독자들이 참여한 오킹챌린지! 1등의 주인공은!? ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 516,865회 좋아요수 5.9천개 싫어요수 68개 게시일 2019. 9. 12.
137 . 썸네일 https://i.ytimg.com/vi/65J7P_BS1vo/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB9VcTz5elDr7JbW5TmHL3IBlpq7A 제목: 군대에서 간부들에게 '다나까' 쓰지 않고 이야기하는 방법. ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 524,558회 좋아요수 6.2천개 싫어요수 97개 게시일 2019. 9. 11.
138 . 썸네일 https://i.ytimg.com/vi/oG3p_f-Td3A/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCKZACosyA3Vn98yctdJSr4XWUtdg 제목: 도로에서 추격전이 일어났습니다..ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 260,581회 좋아요수 3.4천개 싫어요수 69개 게시일 2019. 9. 10.
139 . 썸네일 https://i.ytimg.com/vi/JDPgSYmhpo0/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDT2w7riYQKBRpREDFHLoGHuef1bw 제목: '실버버튼' 받아서 보고나니 충격적이네요..ㅣ주간핫클립ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 134,743회 좋아요수 1.8천개 싫어요수 23개 게시일 2019. 9.

...

268 . 썸네일 https://i.ytimg.com/vi/7bttCUt33dA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCfQs5gM43qyu3q_p1Ql1Gu-rgAMA 제목: 역대급(한때 칭해진) RPG게임에 새로운 직업이 나왔다고...? 오지게 쌔네 ㅋㅋ l 로스트아크 l ㅣ오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 26,785회 좋아요수 259개 싫어요수 3개 게시일 2019. 5. 10.
269 . 썸네일 https://i.ytimg.com/vi/kAhaSSRIipo/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCIMz6-jWwVjauGBJgox8h2NXKs-A 제목: 일타! 강사 오킹이 알려주는 좋은 숙소 고르는 꿀팁!ㅣ국토대장정 27화 8일차 (3/3)ㅣ오킹 TV 카테고리 엔터테인먼트 조회수 조회수 95,882회 좋아요수 944개 싫어요수 17개 게시일 2019. 5. 9.
270 . 썸네일 https://i.ytimg.com/vi/MtVXY8Jm5iM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC_ZxPDC41tZPH8ONWgNagqi-OW5Q 제목: 다시는.... 꺼내지 않을 이야기(2/2) l 오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 597,287회 좋아요수 9.3천개 싫어요수 132개 게시일 2019. 5. 8.
271 . 썸네일 https://i.ytimg.com/vi/tdRAxjfZEI8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBniiKhPBy-4uQwLk6hNie-N2F1Ww 제목: 다시는.... 꺼내지 않을 이야기(1/2) l 오킹TVㅣ 카테고리 엔터테인먼트 조회수 조회수 900,3

301 . 썸네일 https://i.ytimg.com/vi/18xi5ZAtZLk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDttHCPnV7ocxoGLgFVuXg2UI7A3w 제목: 방송하다 앞니가 하나 부러진 이유는??! l 주간핫클립 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 108,291회 좋아요수 1.3천개 싫어요수 19개 게시일 2019. 4. 8.
302 . 썸네일 https://i.ytimg.com/vi/v64Zbe-1iS0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCCRjkuk7SRcXVLv99Owyjq0gDjiw 제목: 만수르 보고 있나? 대한민국의 부를 보여주마 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 206,682회 좋아요수 1.7천개 싫어요수 41개 게시일 2019. 4. 7.
303 . 썸네일 https://i.ytimg.com/vi/obrrcD6MYZk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAK_ijpVHhcHFJL0XylXPOYVf_DMw 제목: 눈 앞에서 160만원 놓쳤습니다... (feat. 탬탬버린, 오퀸) 카테고리 게임 조회수 조회수 142,408회 좋아요수 1.1천개 싫어요수 14개 게시일 2019. 4. 6.
304 . 썸네일 https://i.ytimg.com/vi/ux0RZzuUY4o/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBCOH1gGp6RAF0Kv9kwjQ8CUGtv4w 제목: (혈압주의)내 돈 안돌려주는 건물주 때문에 변호사 상담 받았습니다... l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 365,035회 좋아요수 3.8천개 싫어요수 93개 게시일 2

334 . 썸네일 https://i.ytimg.com/vi/Ng9qy_7CgAo/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBEfa0Zhka8Mi878jDVHacCdSBH1w 제목: 성대모사 3개 1분안에 알려드립니다 l 주간핫클립 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 124,931회 좋아요수 2천개 싫어요수 15개 게시일 2019. 3. 6.
335 . 썸네일 https://i.ytimg.com/vi/S3hlhgyG0r8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDHa-_XHVQu1FLUzAUrVuux1qLrnQ 제목: 동반입대한 친구 벌서는데 놀렸던 썰 핵꿀잼ㅋㅋㅋ l 국토대장정 16화 5일차(4/5) l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 268,663회 좋아요수 2.1천개 싫어요수 51개 게시일 2019. 3. 5.
336 . 썸네일 https://i.ytimg.com/vi/N-VDJEGaaps/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCnhpGgXQCAjdoIYQoSqMEhgiObDQ 제목: 3.1절에 90명 모은 대규모 컨텐츠 제대로 말아먹었다 l 역사 골든벨 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 44,533회 좋아요수 755개 싫어요수 8개 게시일 2019. 3. 4.
337 . 썸네일 https://i.ytimg.com/vi/NovJjn4UUBc/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC7zZU--bZNUm-4GaPWXLYWErW4cw 제목: 3.1절에 시민들과 함께 대한독립만세 외쳤습니다 IN 서대문형무소 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 32,991회 좋

367 . 썸네일 https://i.ytimg.com/vi/dYp6jBRUmlE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBSMHYPKSCF_9LYveIiIRwkwdYoBg 제목: 콜라로 샤워하기 l 피자 먹방 레전드 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 378,010회 좋아요수 5.7천개 싫어요수 44개 게시일 2019. 1. 28.
368 . 썸네일 https://i.ytimg.com/vi/-k8qzu-qch4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBtKtfPmm_0EvofSvNR2_X2k94VZw 제목: 견자희님께 직접듣는 '오병민미워' l 주간핫클립 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 305,419회 좋아요수 2.6천개 싫어요수 32개 게시일 2019. 1. 27.
369 . 썸네일 https://i.ytimg.com/vi/UZUfC-l5Po0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDjOwjbv2fLRa8QnJNmHqYf6-noJg 제목: 주차문제로 싸우다 몸싸움까지?! l 저는 억울합니다 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 371,387회 좋아요수 4.5천개 싫어요수 51개 게시일 2019. 1. 26.
370 . 썸네일 https://i.ytimg.com/vi/kinKSZB1iRM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCUpDi-x9DgNgWFFAJkUfQKZ8AYGw 제목: 대전까지 걸어가면 장어를 공짜로 준다고?!! l 국토대장정2 9화 3일차(3/3) l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 296,279회 좋아요수 2.2천개 싫어요수 35개 게시

400 . 썸네일 https://i.ytimg.com/vi/01GFVZlzD0w/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAlkl-CX9cGflaMn_hb_l4jVFCvRA 제목: 생방도중 여자 시청자의 습격 l 고기먹방 l 미친패션 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 277,163회 좋아요수 2.5천개 싫어요수 23개 게시일 2018. 12. 22.
401 . 썸네일 https://i.ytimg.com/vi/76TKQsXtUOQ/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDba0JIRuScceZuXv3_fRFUcjEUdA 제목: 시급 30만원! 5시간 걸어서 150만원 벌기 l 국토대장정2 1일차 2화 l 이러다 만수르되겠다 l 오킹TV 카테고리 엔터테인먼트 조회수 조회수 194,067회 좋아요수 1.9천개 싫어요수 14개 게시일 2018. 12. 21.
402 . 썸네일 https://i.ytimg.com/vi/37mJFtb-5oY/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDHLK-j-kL-eeh4X7TlqbBBHsdfXw 제목: 야생의 군고구마 먹방 I 장작불 피우기 I 다양한 농작물 소개 I 오킹TV 카테고리 엔터테인먼트 조회수 조회수 95,260회 좋아요수 1.3천개 싫어요수 14개 게시일 2018. 12. 21.
403 . 썸네일 https://i.ytimg.com/vi/CLgGL_qRI-g/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCOhLmLe26THeMD5zDhtxl6axMBlQ 제목: 식당손님한테 다짜고짜 욕먹었습니다 l 대한민국 주차문제 심각 l 오킹TV 카테고리 엔터테인먼트 조회수 조회

In [ ]:
print(len(thumbnail_list),len(video_length_list),len(title_list))

In [ ]:
import copy
video_length_list_copy = copy.deepcopy(video_length_list[:len(thumbnail_list)])
len(video_length_list_copy), len(video_length_list)

In [ ]:
import pandas as pd

insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list_copy
                    }) 
insert_data

In [ ]:
import openpyxl
insert_data.to_excel("오킹.xlsx",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘,튜브몬,박병창

In [ ]:
df